In [1]:
import pandas as pd
import re
from bs4 import BeautifulSoup
import unicodedata

In [18]:

#importing scraping dataset
repub_data_1 = pd.read_csv ('/projects/820ea3ac-497c-43fd-9cb7-abe34292faa8/SC22-BatchA-Blurred-Futures/Data/nishant-republicans-1year.csv')
repub_data_1 = repub_data_1[['body', 'score']] 
repub_data_1

,body,score
0,Serious question. Why do Republicans always ta...,0
1,You think Lebron James wants complete abolishm...,0
2,I thought it was the welfare and govt help\n t...,0
3,Ngl it is a good luck. Did you ever find that ...,1
4,Why? He’s right,0
...,...,...
24211,Gunman kills 19 children in Texas school rampa...,1
24212,Gunman kills 19 children in Texas school rampa...,1
24213,"\n""Please note that this is a **Republican** s...",1
24214,"\n""Please note that this is a **Republican** s...",1


In [19]:
#importing kaggle dataset and removing unwanted columns
repub_data_2 = pd.read_csv ('/projects/820ea3ac-497c-43fd-9cb7-abe34292faa8/SC22-BatchA-Blurred-Futures/Data/kaggle_dataset.csv')
repub_data_2 = repub_data_2[repub_data_2['Political Lean'] == 'Conservative']
repub_data_2 = repub_data_2[['Title', 'Score']]
repub_data_2 = repub_data_2[repub_data_2['Title'] != '[removed]']
repub_data_2 = repub_data_2[repub_data_2['Score']>=3]
repub_data_2 = repub_data_2.replace(to_replace ='\n', value = ' ', regex = True)
repub_data_2.columns = ['body', 'score']
repub_data_2

,body,score
8334,"You fired our troops, Don’t ever act like you ...",13
8336,BREAKING: Ben Shapiro rebuts Joe Biden's 'path...,10
8337,BREAKING: Biden heckled over 13 killed soldier...,6
8338,State of the Union: Republicans hammer Biden f...,3
8339,State of the Union: Republicans hammer Biden f...,5
...,...,...
12839,"""You folks need to lay off of the theory and a...",9
12840,"Carson's rejoinders to criticisms of his ""Stud...",10
12841,This Is the Obama-Aurora Billboard Everyone's ...,12
12842,Questions about The School Sucks Podcast,6


In [5]:
extra = '\n"Please note that this is a **Republican** subreddit. Please mind our [rules](/r/Republicans Rules](https://www.reddit.com/r/Republicans/about/rules/). Trolls and anyone who violates the rules stated in this message may be banned." \n\n*I am a bot, and this action was performed automatically. Please [contact the moderators of this subreddit](/message/compose/?to=/r/republicans) if you have any questions or concerns.*'

In [20]:
#removing unwanted columns
repub_data_1 = repub_data_1[repub_data_1['body'] != extra]
repub_data_1 = repub_data_1[repub_data_1['body'] != '[removed]']
repub_data_1 = repub_data_1[repub_data_1['score']>=2]
repub_data_1 = repub_data_1.replace(to_replace ='\n', value = ' ', regex = True)
repub_data_1

,body,score
7,"Ngl, Trump would look like a f****** boss if h...",2
10,This photo and its caption are self-evident. ...,3
11,Love Steven Crowder!!!!💕,2
12,"IMO, if she wants to represent our country on ...",2
16,Clever- love the “photo”- perfect.,2
...,...,...
24195,https://www.businessinsider.com/us-systemic-ra...,2
24198,Are you saying non-citizen immigrants are voti...,3
24200,Trump is going to cause us to lose 2024 and he...,2
24205,"Quit buying Nike, simple",2


In [21]:
#combining dataframes
frames = [repub_data_1, repub_data_2]
repub_data = pd.concat(frames)
repub_data['body'][24]

'The chocolate one is the best bar imo'

In [22]:
#lowercasing
repub_data['body'] = repub_data['body'].apply(lambda x: str(x).lower())

In [23]:
#no more emails
repub_data['body'] = repub_data['body'].apply(lambda x: re.sub(r'([a-z0-9+._-]+@[a-z0-9+._-]+\.[a-z0-9+_-]+)',"", x))
repub_data.shape

(6749, 2)

In [24]:
#removes url
repub_data['body'] = repub_data['body'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , x))
repub_data.shape

(6749, 2)

In [25]:
#figure out how to keep ./-,
repub_data['body'] = repub_data['body'].apply(lambda x: re.sub(r'[^\w.\',;!?]+', " ", x))
repub_data

,body,score
7,"ngl, trump would look like a f boss if he did....",2
10,this photo and its caption are self evident. i...,3
11,love steven crowder!!!!,2
12,"imo, if she wants to represent our country on ...",2
16,clever love the photo perfect.,2
...,...,...
12839,you folks need to lay off of the theory and a...,9
12840,carson's rejoinders to criticisms of his studi...,10
12841,this is the obama aurora billboard everyone's ...,12
12842,questions about the school sucks podcast,6


In [26]:
repub_data = repub_data[repub_data['body'] != '']
repub_data.shape

(6732, 2)

In [27]:
#removes multiple spaces
repub_data['body'] = repub_data['body'].apply(lambda x: ' '.join(x.split()))

/tmp/ipykernel_4884/984530229.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  repub_data['body'] = repub_data['body'].apply(lambda x: ' '.join(x.split()))


In [28]:
#removes html
repub_data['body'] = repub_data['body'].apply(lambda x: BeautifulSoup(x, 'html.parser').get_text().strip())

/tmp/ipykernel_4884/3590185498.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  repub_data['body'] = repub_data['body'].apply(lambda x: BeautifulSoup(x, 'html.parser').get_text().strip())


In [29]:
#removes Accents
def remove_accented_chars(x):
    x = unicodedata.normalize('NFKD', x).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return x
repub_data['body'] = repub_data['body'].apply(lambda x: remove_accented_chars(x))

/tmp/ipykernel_4884/3254416418.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  repub_data['body'] = repub_data['body'].apply(lambda x: remove_accented_chars(x))


In [30]:
#filters out word counts
repub_data['word_counts'] = repub_data['body'].apply(lambda x: len(str(x).split()))
repub_data = repub_data[repub_data['word_counts']>8]
repub_data

/tmp/ipykernel_4884/2475686139.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  repub_data['word_counts'] = repub_data['body'].apply(lambda x: len(str(x).split()))


,body,score,word_counts
7,"ngl, trump would look like a f boss if he did....",2,28
10,this photo and its caption are self evident. i...,3,92
12,"imo, if she wants to represent our country on ...",2,122
21,the biden administration doesn t want them com...,3,15
26,everything about this current white house admi...,2,11
...,...,...,...
12832,us is the worst police state in the world by t...,6,12
12837,how would restitution be enforced in an ancap ...,3,9
12839,you folks need to lay off of the theory and ac...,9,14
12840,carson's rejoinders to criticisms of his studi...,10,12


In [31]:
repub_data = repub_data[repub_data['body'] != '[deleted]']
repub_data.shape

(4553, 3)

In [33]:
path='/projects/820ea3ac-497c-43fd-9cb7-abe34292faa8/SC22-BatchA-Blurred-Futures/Data/'

In [34]:
repub_data.to_csv(path+'clean-republican-data.csv')